# Identify & Track Marine Heatwaves on _Unstructured Grid_ using `spot_the_blOb`

## Processing Steps:
1. Fill spatial holes in the binary data, using `dask_image.ndmorph` -- up to `R_fill` cells in radius.
2. Fill gaps in time -- permitting up to `T_fill` missing time slices, while keeping the same blob ID.
3. Filter out small objects -- area less than the bottom `area_filter_quartile` of the size distribution of objects.
4. Identify objects in the binary data, using `dask_image.ndmeasure`.
5. Connect objects across time, applying the following criteria for splitting, merging, and persistence:
    - Connected Blobs must overlap by at least fraction `overlap_threshold` of the smaller blob.
    - Merged Blobs retain their original ID, but partition the child blob based on the parent of the _nearest-neighbour_ cell. 
6. Cluster and reduce the final object ID graph using `scipy.sparse.csgraph.connected_components`.
7. Map the tracked objects into ID-time space for convenient analysis.

N.B.: Exploits parallelised `dask` operations with optimised chunking using `flox` for memory efficiency and speed \
N.N.B.: This example using 40 years of _daily_ outputs at 5km resolution on an Unstructured Grid (15 million cells) using 32 cores takes 
- Full Split/Merge Thresholding & Merge Tracking:  ~40 minutes

In [1]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import spot_the_blOb as blob
import spot_the_blOb.helper as hpc

In [ ]:
# Start Dask Cluster
#  N.B.: Need ~ 8 GB per worker (for 5km data // 15 million points)
client = hpc.StartLocalCluster(n_workers=64, n_threads=1)

Memory per Worker: 7.87 GB
Hostname is  l40329
Forward Port = l40329:8787
Dashboard Link: localhost:8787/status


2025-02-13 12:20:04,770 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:35407 (pid=4008375) exceeded 95% memory budget. Restarting...
2025-02-13 12:20:06,098 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:38275 (pid=4008172) exceeded 95% memory budget. Restarting...
2025-02-13 12:20:06,125 - distributed.nanny - WARNING - Restarting worker
2025-02-13 12:20:06,197 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:40111 (pid=4008184) exceeded 95% memory budget. Restarting...
2025-02-13 12:20:06,201 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:44557 (pid=4008196) exceeded 95% memory budget. Restarting...
2025-02-13 12:20:06,265 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:39505 (pid=4008277) exceeded 95% memory budget. Restarting...
2025-02-13 12:20:06,272 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:38483 (pid=4008293) exceeded 95% memory budget. Restarting...
2025-02-13 12:20:06,276 - distributed.

In [3]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'extreme_events_binary_unstruct.zarr'
chunk_size = {'time': 4, 'ncells': -1}
ds = xr.open_zarr(str(file_name), chunks={}).isel(time=slice(0, 128)).chunk(chunk_size)

In [4]:
# Tracking Parameters

drop_area_quartile = 0.8  # Remove the smallest 80% of the identified blobs
hole_filling_radius = 32  # Fill small holes with radius < 32 elements, i.e. ~100 km
time_gap_fill = 2         # Allow gaps of 4 days and still continue the blob tracking with the same ID
allow_merging = True      # Allow blobs to split/merge. Keeps track of merge events & unique IDs.
overlap_threshold = 0.5   # Overlap threshold for merging blobs. If overlap < threshold, blobs keep independent IDs.
nn_partitioning = True    # Use new NN method to partition merged children blobs. If False, reverts to old method of Di Sun et al. 2023.

In [ ]:
# SpOt & Track the Blobs & Merger Events

tracker = blob.Spotter(ds.extreme_events, ds.mask, R_fill=hole_filling_radius, T_fill = time_gap_fill, area_filter_quartile=drop_area_quartile, 
                       allow_merging=allow_merging, overlap_threshold=overlap_threshold, nn_partitioning=nn_partitioning, 
                       xdim='ncells',                 # Need to tell spot_the_blOb the new Unstructured dimension
                       unstructured_grid=True,        # Use Unstructured Grid
                       neighbours=ds.neighbours,      # Connectivity array for the Unstructured Grid Cells
                       cell_areas=ds.cell_areas,      # Cell areas for each Unstructured Grid Cell
                       verbosity=1)                   # Choose Verbosity Level (0=None, 1=Basic, 2=Advanced/Timing)

blobs = tracker.run(return_merges=False)

blobs

Finished Constructing the Sparse Dilation Matrix.
Finished Filling Spatial Holes
Finished Filling Spatio-temporal Holes.
Finished Filtering Small Blobs.
Finished Blob Identification.
Finished Making Blobs Globally Unique.
Finished Calculating Blob Properties.
Finished Finding Overlapping Blobs.
Processing Parallel Iteration 1 with 27 Merging Blobs...
Processing Parallel Iteration 2 with 20 Merging Blobs...


2025-02-13 12:12:22,739 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.61 GiB -- Worker memory limit: 7.61 GiB


Processing Parallel Iteration 3 with 5 Merging Blobs...


2025-02-13 12:14:31,781 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.21 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:14:32,729 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.03 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:14:32,881 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

Processing Parallel Iteration 4 with 4 Merging Blobs...


...

2025-02-13 12:19:14,832 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 6.09 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:19:14,932 - distributed.worker.memory - WARNING - Worker is at 79% memory usage. Resuming worker. Process memory: 6.07 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:19:14,941 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 6.22 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:19:15,032 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 6.23 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:19:15,804 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 6.10 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:19:15,850 - distributed.worker.memory - WARNING - Worker is at 59% memory usage. Resuming worker. Process memory: 4.51 GiB -- Worker mem

Processing Parallel Iteration 6 with 1 Merging Blobs...


2025-02-13 12:20:01,204 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 6.14 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:20:02,013 - distributed.worker.memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 6.24 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:20:02,051 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 6.16 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:20:02,088 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 6.40 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 12:20:02,163 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memo

KilledWorker: Attempted to run task ('process_chunk-5d53519d6c79f33fbd7c89a0c1c6136d', 19) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:42397. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

2025-02-13 13:00:22,216 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 6.22 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 13:00:22,822 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.01 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 13:00:23,391 - distributed.worker.memory - WARNING - Worker is at 63% memory usage. Resuming worker. Process memory: 4.84 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 13:00:25,630 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 6.39 GiB -- Worker memory limit: 7.61 GiB
2025-02-13 13:00:26,124 - distributed.worker.memory - WARNING - Worker is at 59% memory usage. Resuming worker. Process memory: 4.49 GiB 

In [ ]:
blobs = blobs.compute() 

In [ ]:
# Save Tracked Blobs to `zarr` for more efficient parallel I/O

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'MHWs_tracked_unstruct.zarr'
blobs.to_zarr(file_name, mode='w')